In [2]:
import sys
import csv
import os
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.runnables import RunnableLambda
from tqdm import tqdm

sys.path.append('utils')

import openai_utils

In [3]:
def split_text_by_sections(input_file, chunk_size=75000):
    # Leer el archivo completo
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    chunks = []
    current_chunk = ""

    for line in lines:
        # Si una línea empieza con 'Variables en la carpeta', se considera una nueva sección
        if line.startswith("Variables en la carpeta"):
            # Si el chunk actual supera el tamaño límite, se guarda y se inicia un nuevo chunk
            if len(current_chunk) > chunk_size:
                chunks.append(current_chunk)
                current_chunk = ""

        # Añadir la línea al chunk actual
        current_chunk += line

    # Añadir el último chunk si tiene contenido
    if current_chunk:
        chunks.append(current_chunk)

    return chunks



In [4]:
# Uso de la función
input_file = 'data/variables_por_carpeta.txt'  # Ruta al archivo de texto grande
chunks = split_text_by_sections(input_file)

# Mostrar la cantidad de fragmentos y el tamaño de cada fragmento
print(f"Se han creado {len(chunks)} fragmentos.")
for i, chunk in enumerate(chunks):
    print(f"Fragmento {i + 1} - Tamaño: {len(chunk)} caracteres")


Se han creado 10 fragmentos.
Fragmento 1 - Tamaño: 75311 caracteres
Fragmento 2 - Tamaño: 93999 caracteres
Fragmento 3 - Tamaño: 75009 caracteres
Fragmento 4 - Tamaño: 75383 caracteres
Fragmento 5 - Tamaño: 76922 caracteres
Fragmento 6 - Tamaño: 75813 caracteres
Fragmento 7 - Tamaño: 75253 caracteres
Fragmento 8 - Tamaño: 76244 caracteres
Fragmento 9 - Tamaño: 75409 caracteres
Fragmento 10 - Tamaño: 48332 caracteres


In [5]:
chunks[0]

"Variables en la carpeta 'AWS_Transversaltech_SD/Comfama_Digital/form':\n - id\n - form_name\n\nVariables en la carpeta 'AWS_Transversaltech_SD/Comfama_Digital/form_formquestion':\n - form_id\n - question_id\n\nVariables en la carpeta 'AWS_Transversaltech_SD/Comfama_Digital/form_question':\n - id\n - id_question\n - question\n\nVariables en la carpeta 'AWS_Transversaltech_SD/Comfama_Digital/form_response':\n - id\n - answer\n - form_id\n - question_id\n - user_id\n\nVariables en la carpeta 'AWS_Transversaltech_SD/Comfama_Digital/user':\n - id\n - address\n - cellphone\n - city\n - birth_date\n - email\n - first_name\n - gender\n - document_number\n - document_type\n - last_name\n - phone\n\nVariables en la carpeta 'AWS_Transversaltech_SD/Comfama_Digital/user_form':\n - form_id\n - user_id\n - reply_date\n - consecutive\n\nVariables en la carpeta 'AdmonSesionesMentoriaFinanciera/DatosPersonales/_delta_log':\n - appId\n - version\n - lastUpdated\n - path\n - key\n - value\n - size\n - mo

In [6]:
aoi_client = openai_utils.AzureOpenAIFunctions()
aoi_client.llm.invoke("Hola")

AIMessage(content='¡Hola! ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-ffa0c991-0686-40bb-a34c-11c41438d131-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18})

In [7]:
# Definir las plantillas de mensajes del chat
system_template = "A partir de la siguiente lista de variables en distintas carpetas, genera un resumen que explique brevemente el contenido y propósito de las variables en cada carpeta."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = """
Texto a procesar: {variables}

"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combinar las plantillas en un solo prompt de chat
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

def output_parser(response):
    return response["text"]

# Create the LLM chain
chain = LLMChain(llm=aoi_client.llm, prompt=chat_prompt) | RunnableLambda(output_parser)

C:\Users\SebastianCarmonaEstr\AppData\Local\Temp\ipykernel_25160\1242238990.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=aoi_client.llm, prompt=chat_prompt) | RunnableLambda(output_parser)


In [8]:
resumenes = []

for chunk in tqdm(chunks):
    response = chain.invoke(chunk)
    resumenes.append(response)

100%|██████████| 10/10 [05:11<00:00, 31.15s/it]


In [9]:
resumenes

['# Resumen del contenido y propósito de las variables en cada carpeta\n\n## AWS_Transversaltech_SD/Comfama_Digital\n\n### form\n- **Variables:**\n  - `id`\n  - `form_name`\n\n- **Propósito:**\n  - Almacenar información básica sobre los formularios disponibles, como su identificador y nombre.\n\n### form_formquestion\n- **Variables:**\n  - `form_id`\n  - `question_id`\n\n- **Propósito:**\n  - Relacionar formularios con sus respectivas preguntas mediante identificadores.\n\n### form_question\n- **Variables:**\n  - `id`\n  - `id_question`\n  - `question`\n\n- **Propósito:**\n  - Gestionar las preguntas de los formularios, asignando un identificador único a cada pregunta.\n\n### form_response\n- **Variables:**\n  - `id`\n  - `answer`\n  - `form_id`\n  - `question_id`\n  - `user_id`\n\n- **Propósito:**\n  - Almacenar las respuestas de los usuarios a las preguntas de los formularios, incluyendo detalles de identificación del formulario, la pregunta y el usuario.\n\n### user\n- **Variables:*

**Resumen General Detallado del Contenido y Propósito de las Variables en las Carpetas**

---

### **1. Formularios Digitales y Gestión de Usuarios**
**Carpeta:** `AWS_Transversaltech_SD/Comfama_Digital`

- **Subcarpetas y Variables:**
  - **form:**
    - `id`
    - `form_name`
  - **form_formquestion:**
    - `form_id`
    - `question_id`
  - **form_question:**
    - `id`
    - `id_question`
    - `question`
  - **form_response:**
    - `id`
    - `answer`
    - `form_id`
    - `question_id`
    - `user_id`
  - **user:**
    - `id`
    - `first_name`
    - `last_name`
    - `email`
    - `document_number`
  - **user_form:**
    - `form_id`
    - `user_id`
    - `reply_date`
    - `consecutive`

- **Propósito:**
  - Facilitar la creación y administración de formularios digitales.
  - Capturar y relacionar las respuestas de los usuarios con las preguntas y formularios correspondientes.
  - Mantener un registro detallado de los usuarios que interactúan con los formularios.

---

### **2. Sesiones de Mentoría Financiera**
**Carpeta:** `AdmonSesionesMentoriaFinanciera`

- **Subcarpetas y Variables:**
  - **DatosPersonales:**
    - `NúmeroDeDocumento`
    - `NombreEmpresa`
    - `Celular1`
    - `Email`
    - `Edad`
    - `Ocupación`
  - **EstudioCredito:**
    - `EstadoCréditoValue`
    - `NivelDeEndeudamiento`
    - `Salario`
    - `EgresosAntesDelCrédito`
  - **GestionDeLlamadaDeBienvenida:**
    - `DeseaParticiparValue`
    - `EstadoDeLaLlamadaValue`
    - `ExpectativasDelUsuario`
  - **InformacionPresupuesto:**
    - `Salario`
    - `Arrendamiento`
    - `Alimentación`
    - `Transporte`
    - `Ahorro`
  - **Seguimiento:**
    - `EstadoMentoríaValue`
    - `PuntajeDataInicial`
    - `MentorAsignadoValue`
    - `PuntajeDeDiagnósticoFinalOCDE`

- **Propósito:**
  - Gestionar la información personal y financiera de los participantes en programas de mentoría financiera.
  - Realizar seguimiento al progreso y eficacia de las sesiones de mentoría.
  - Evaluar el impacto de las mentorías en el bienestar financiero de los usuarios.

---

### **3. Datos Territoriales y Demográficos**
**Carpetas:**
- `Cerebro_Territorial`
- `Corporativo/ModeloDemografico`

- **Variables:**
  - Indicadores estadísticos regionales como `empleo`, `educación`, `delincuencia`, `demografía`, `calidad de vida`.
  - Proyecciones poblacionales con variables como `Año`, `Sexo`, `RangoEdad`.
  - Datos de afiliados como `genero`, `edad`, `tipo_de_afiliado`, `nuevos`, `retirados`.

- **Propósito:**
  - Proporcionar información para la planificación urbana y el desarrollo social.
  - Analizar tendencias demográficas y su impacto en la organización.
  - Apoyar la toma de decisiones basada en datos estadísticos territoriales.

---

### **4. Facturación y Suscripciones**
**Carpeta:** `Chargebee`

- **Variables:**
  - **customers:**
    - `customer_id`
    - `email`
    - `first_name`
    - `last_name`
  - **subscriptions:**
    - `subscription_id`
    - `plan_id`
    - `status`
    - `current_term_start`
  - **invoices:**
    - `invoice_id`
    - `amount_due`
    - `amount_paid`
    - `status`

- **Propósito:**
  - Gestionar las relaciones con los clientes y sus planes de suscripción.
  - Registrar y controlar las facturas emitidas, pagos y estados financieros.
  - Mantener un historial detallado de transacciones para auditorías y análisis financieros.

---

### **5. Conexión Empleo y Bots**
**Carpeta:** `Conexion_Empleo_Bots`

- **Variables:**
  - **CandidatosRemitidosBot**, **CandidatosVinculadosBot:**
    - `Prop_0`, `Prop_1`, ..., `Prop_15`
  - **EmpresasVinculadasBot**, **VacantesRegistradasBot:**
    - Variables genéricas que representan diversos atributos de empresas y vacantes.

- **Propósito:**
  - Automatizar y optimizar el proceso de reclutamiento mediante bots.
  - Facilitar el emparejamiento entre candidatos y vacantes disponibles.
  - Gestionar la información de empresas que ofrecen oportunidades laborales.

---

### **6. Informes Contables y Financieros**
**Carpeta:** `ContabilidadInformeLey`

- **Variables:**
  - **Proyectos y Desembolsos:**
    - `Desembolso`
    - `Reintegro`
    - `Proyecto`
    - `Rendimiento_Mes`
  - **Tablas de Referencia:**
    - Códigos y descripciones de cuentas.
    - Tipos de inversión.
    - Fuentes de recursos.
  - **Consolidado de Cuentas:**
    - Detalles de cuentas de inversión y consolidaciones.

- **Propósito:**
  - Llevar un control detallado de los movimientos financieros asociados a proyectos específicos.
  - Asegurar el cumplimiento de normas contables y legales en la gestión financiera.
  - Facilitar la elaboración de informes financieros y auditorías.

---

### **7. Gestión Escolar y Estudiantil**
**Carpeta:** `Cosmo_Schools`

- **Variables:**
  - **Estudiantes:**
    - `nombres`
    - `apellidos`
    - `documento`
    - `grado`
    - `sede`
  - **Finanzas:**
    - `cuentas`
    - `centros_de_costos`
    - `movimientos_mensuales`
  - **Mentores:**
    - `nombre`
    - `contacto`
    - `asignaturas`
  - **Capacidad Sedes:**
    - `capacidad`
    - `meta_matricula`

- **Propósito:**
  - Administrar eficientemente los recursos educativos y financieros de las instituciones.
  - Hacer seguimiento académico y de bienestar de los estudiantes.
  - Planificar y gestionar el personal docente y administrativo.

---

### **8. Autorizaciones Médicas y Recaudos**
**Carpeta:** `Crome`

- **Variables:**
  - **Autorizaciones:**
    - `solicitante`
    - `fecha_solicitud`
    - `servicio`
    - `estado`
  - **Recaudos:**
    - Detalles de cobros asociados a servicios médicos.

- **Propósito:**
  - Gestionar las autorizaciones de servicios médicos y su facturación.
  - Asegurar que los servicios prestados estén debidamente autorizados y cobrados.
  - Facilitar el seguimiento y control de los servicios de salud ofrecidos.

---

### **9. Gestión de Ventas y Asesores**
**Carpeta:** `Cubo_De_Ventas`

- **Variables:**
  - **Aliado_de_negocio**, **Asesores_Pedido_Sales:**
    - `user_id`
    - `user_name`
    - `sector`
  - **Facturación y Entregas:**
    - `documento_comercial`
    - `monto`
    - `fecha`
    - `estado`
  - **Insumos y Metas:**
    - `metas_ventas`
    - `portafolio`
    - `productos`

- **Propósito:**
  - Hacer seguimiento y análisis del desempeño de ventas y cumplimiento de metas.
  - Gestionar relaciones con clientes y aliados de negocio.
  - Optimizar estrategias comerciales y de marketing.

---

### **10. Eventos Culturales y Participación**
**Carpeta:** `Cultura`

- **Variables:**
  - **Agenda**, **Eventos**, **Participantes:**
    - `fecha_evento`
    - `ubicacion`
    - `participantes`
    - `costo`
  - **Subsidios Aplicados:**
    - `categoria_afiliacion`
    - `valor_pagado`
    - `valor_subsidiado`

- **Propósito:**
  - Promover y gestionar actividades culturales y educativas.
  - Facilitar el acceso a eventos mediante subsidios y programas de apoyo.
  - Registrar y analizar la participación en eventos para futuras planificaciones.

---

### **11. Conectividad y Uso de Servicios WiFi**
**Carpeta:** `DATAWIFI`

- **Variables:**
  - **Usuarios Únicos**, **DataWifi:**
    - `sesiones`
    - `zona_conexion`
    - `duracion`
    - `dispositivo`
    - `email`
    - `telefono`

- **Propósito:**
  - Monitorear y mejorar la calidad del servicio WiFi ofrecido.
  - Analizar patrones de uso para optimizar la infraestructura y cobertura.
  - Generar estrategias de marketing basadas en la interacción de los usuarios.

---

### **12. Desarrollo Empresarial**
**Carpeta:** `Desarollo_Empresarial`

- **Variables:**
  - **Gestión Contratos:**
    - `fecha_inicio`
    - `fecha_fin`
    - `valor_contrato`
    - `estado`
  - **Usos Empresas:**
    - `sede`
    - `mes`
    - `tipo_servicio`

- **Propósito:**
  - Gestionar las relaciones contractuales con empresas.
  - Monitorear la utilización de servicios para identificar oportunidades de mejora.
  - Facilitar la toma de decisiones estratégicas en el área de desarrollo empresarial.

---

### **13. Diagnóstico de Bienestar y Encuestas**
**Carpeta:** `DiagnosticoBienestar`

- **Variables:**
  - **Encuestas:**
    - `datos_personales`
    - `situacion_familiar`
    - `salud`
    - `habitos`
  - **DiagnosticoBienestar_Regiones:**
    - Datos específicos por regiones.
  - **RutaProgreso:**
    - Seguimiento del progreso de los participantes.

- **Propósito:**
  - Evaluar el bienestar general de los usuarios y detectar áreas de mejora.
  - Personalizar programas y servicios según las necesidades identificadas.
  - Medir el impacto de las iniciativas de bienestar implementadas.

---

**Conclusión General:**

Los datos y variables recopilados en estas carpetas son fundamentales para el funcionamiento integral de la organización. Permiten:

- **Toma de Decisiones Informada:** Proporcionan información detallada y actualizada en diversas áreas, facilitando decisiones estratégicas y operativas.

- **Mejora Continua de Servicios:** Ayudan a ajustar y mejorar los servicios ofrecidos para satisfacer mejor las necesidades de usuarios y clientes.

- **Eficiencia Operativa:** La gestión centralizada de información reduce redundancias y errores, aumentando la eficiencia en procesos internos.

- **Cumplimiento Legal y Normativo:** La adecuada gestión de datos financieros, contables y de salud asegura el cumplimiento de regulaciones y estándares legales.

- **Fortalecimiento de Relaciones:** La información sobre clientes, usuarios y aliados comerciales ayuda a fortalecer las relaciones y crear oportunidades de crecimiento conjunto.

---

**Nota:** Es esencial mantener prácticas sólidas de seguridad y privacidad de datos para proteger la información personal y sensible, cumpliendo con las regulaciones aplicables.

